In [1]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from njl_corf import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]


In [2]:
import json
from typing import TextIO
import re

In [3]:
%autoreload 2

In [4]:
from njl_corf import wrc27_views

In [25]:
def wrap_frequency_ranges(input_text: str) -> str:
    """
    Finds frequency ranges in the input text and wraps them in LaTeX \mbox{}.
    A frequency range is defined as "<f-low>-<f-high> <units>" where:
    - <f-low> and <f-high> use white spaces as thousands separators and may contain decimals.
    - <units> is one of kHz, MHz, or GHz.
    """
    # Regex to match the frequency range, allowing for decimals
    freq_range_pattern = r"\b(\d{1,3}(?: \d{3})*(?:\.\d+)?)-(\d{1,3}(?: \d{3})*(?:\.\d+)?)\s*(kHz|MHz|GHz)\b"

    # Substitute the matched frequency range with the LaTeX \mbox{}
    def replacer(match):
        f_low, f_high, units = match.groups()
        return f"\\mbox{{{f_low}-{f_high} {units}}}"

    # Apply the substitution to the input text
    return re.sub(freq_range_pattern, replacer, input_text)

In [34]:
def format_frequency_ranges(input_text: str) -> str:
    """
    Finds frequency ranges in the input text where the separator is an en-dash (\u2013),
    and reformats them for LaTeX:
    - Wraps the frequency range in \mbox{}.
    - Replaces the en-dash (\u2013) with "--".
    - Replaces the space before the unit with a LaTeX half-space ("\,"), preserving units.
    """
    # Regex to match the frequency range with an en-dash (\u2013)
    freq_range_pattern = r"\b(\d+(?:\.\d+)?)\u2013(\d+(?:\.\d+)?)\s*(kHz|MHz|GHz)\b"

    # Substitute the matched frequency range with the formatted version
    def replacer(match):
        f_low, f_high, units = match.groups()
        return f"\\mbox{{{f_low}--{f_high}\,{units}}}"

    # Apply the substitution to the input text
    return re.sub(freq_range_pattern, replacer, input_text)

In [35]:
def clean_ai_text(text: str) -> str:
    """Do things to clean up the text and make it LaTeX ready"""
    return wrap_frequency_ranges(text)

In [36]:
def clean_ai_title(text: str) -> str:
    """Do things to clean up the text and make it LaTeX ready"""
    return format_frequency_ranges(text)
    nbs = "\u00A0"
    dash = "\u2013"
    for unit in ["kHz", " MHz", " GHz"]:
        text = text.replace(f"{nbs}{unit}", f"~{unit}")
    text = text.replace(dash, "--")
    return text

In [37]:
agenda_items = wrc27_views.get_ai_info()

In [38]:
with open("agenda_items.json") as json_file:
    references = json.load(json_file)

In [39]:
print(references.keys())

dict_keys(['27_1.1', '27_1.10', '27_1.11', '27_1.12', '27_1.13', '27_1.14', '27_1.15', '27_1.16', '27_1.17', '27_1.18', '27_1.19', '27_1.2', '27_1.3', '27_1.4', '27_1.5', '27_1.6', '27_1.7', '27_1.8', '27_1.9', '31_2.1', '31_2.10', '31_2.11', '31_2.12', '31_2.13', '31_2.14', '31_2.2', '31_2.3', '31_2.4', '31_2.5', '31_2.6', '31_2.7', '31_2.8', '31_2.9'])


In [42]:
def dump_wrc27_ai(number: int, output_file: TextIO):
    """Write information into the file"""
    details = references[f"27_1.{number}"]
    code = "AI" + chr(number - 1 + ord("a"))
    lines = []
    # Do the section title
    lines.append(
        r"\newcommand{"
        + "\\"
        + code
        + r"Title}{AI-1."
        + str(number)
        + ": "
        + clean_ai_title(details["Short title"])
        + r"}"
    )
    # Now the figure
    lines.append(
        r"\newcommand{"
        + "\\"
        + code
        + r"Figure}{/users/livesey/corf/njl-corf/sandbox/wrc27-views-sandbox/"
        r"specific-ai-plots-no-legend/SpecificAI-WRC-"
        + "27 AI-1."
        + str(number)
        + r".pdf}"
    )
    lines.append(
        r"\newcommand{"
        + "\\"
        + code
        + r"Text}{"
        + clean_ai_text(details["Title"])
        + r"}"
    )
    # Save it
    output_file.writelines(line + "\n" for line in lines)

In [43]:
with open(
    "summary-tabloids/summary-tabloid-content-wrc-27.tex", "w", encoding="utf-8"
) as output_file:
    for number in range(1, 20):
        dump_wrc27_ai(number, output_file=output_file)